<a href="https://colab.research.google.com/github/saisumanthkorada/S2G/blob/main/S2G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install openpyxl


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import split, col,length,right,expr,regexp_extract,regexp_replace,trim,explode,first,lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
movies_explodedB2S = spark.read.format('csv')\
.option('header','True')\
.option('Inferschema','True')\
.load('/content/drive/MyDrive/Silver_Layer/movies_exploded.csv')

In [6]:
movies_explodedB2S.show()

+-------+--------------------+---------+----+
|movieId|               title|  generes|year|
+-------+--------------------+---------+----+
|      1|           Toy Story|Adventure|1995|
|      1|           Toy Story|Animation|1995|
|      1|           Toy Story| Children|1995|
|      1|           Toy Story|   Comedy|1995|
|      1|           Toy Story|  Fantasy|1995|
|      2|             Jumanji|Adventure|1995|
|      2|             Jumanji| Children|1995|
|      2|             Jumanji|  Fantasy|1995|
|      3|    Grumpier Old Men|   Comedy|1995|
|      3|    Grumpier Old Men|  Romance|1995|
|      4|   Waiting to Exhale|   Comedy|1995|
|      4|   Waiting to Exhale|    Drama|1995|
|      4|   Waiting to Exhale|  Romance|1995|
|      5|Father of the Bri...|   Comedy|1995|
|      6|                Heat|   Action|1995|
|      6|                Heat|    Crime|1995|
|      6|                Heat| Thriller|1995|
|      7|             Sabrina|   Comedy|1995|
|      7|             Sabrina|  Ro

In [7]:
RatingsB2S = spark.read.format('csv')\
.option('header','True')\
.option('Inferschema','True')\
.load('/content/drive/MyDrive/Silver_Layer/ratings_transformed.csv')

In [9]:
RatingsB2S.show()

+-------+--------+------+----------+--------------------+
|user_id|movie_id|rating|      date|          movie_name|
+-------+--------+------+----------+--------------------+
|      1|       1|   4.0|2008-52-03|           Toy Story|
|      1|     110|   4.0|2008-04-05|          Braveheart|
|      1|     158|   4.0|2008-31-03|              Casper|
|      1|     260|   4.5|2008-00-03|Star Wars: Episod...|
|      1|     356|   5.0|2008-58-03|        Forrest Gump|
|      1|     381|   3.5|2008-41-03|When a Man Loves ...|
|      1|     596|   4.0|2008-32-03|           Pinocchio|
|      1|    1036|   5.0|2008-07-03|            Die Hard|
|      1|    1049|   3.0|2008-41-03|Ghost and the Dar...|
|      1|    1066|   4.0|2008-29-03|      Shall We Dance|
|      1|    1196|   3.5|2008-04-03|Star Wars: Episod...|
|      1|    1200|   3.5|2008-11-03|              Aliens|
|      1|    1210|   4.5|2008-00-03|Star Wars: Episod...|
|      1|    1214|   4.0|2008-20-03|               Alien|
|      1|    1

In [10]:
RatingsB2S = RatingsB2S.withColumnRenamed("movie_id","movieId")
RatingsB2S = RatingsB2S.withColumnRenamed("date","date_rated")
movies_explodedB2S = movies_explodedB2S.withColumnRenamed("year","releaseyear")

In [26]:
from pyspark.sql.functions import col, avg,count,round

joined_table = movies_explodedB2S\
    .join(RatingsB2S, on="movieId", how="left")\
    .groupBy( "generes")\
    .agg(round(avg("rating"),2).alias("average_rating"),count("user_id").alias("Number of users"))\
    .orderBy(col("average_rating"),col("Number of users"), ascending=False)


In [27]:
joined_table.show()

+---------+--------------+---------------+
|  generes|average_rating|Number of users|
+---------+--------------+---------------+
|Film-Noir|          3.92|         318917|
|      War|           3.8|        1697649|
+---------+--------------+---------------+
only showing top 2 rows

